# Python for Learning AI Week 2: TCP Server

Welcome to Week 2's exploration of port communication in Python! This notebook focuses on the fundamentals of building a TCP server. Understanding how TCP servers work is essential for creating networked applications, APIs, and AI-powered services that exchange data reliably.

> **Looking for HTTP servers?** Check out the companion notebook `002_3_http_server.ipynb` where we build on these ideas to serve web requests.

## What You'll Learn
1. Understanding ports and network communication
2. Creating TCP servers that listen on ports
3. Handling client connections and messages
4. Monitoring active connections on your machine

## Prerequisites
- Basic Python knowledge ([covered in Week 1](../week1/week1_python_basics.ipynb))
- Understanding of functions and error handling

Let's start by confirming the required packages are available:



In [1]:
# Check if required packages are installed
try:
    import socket  # For creating network connections
    import time    # For time-related functions
    import psutil  # For system and process utilities
    print("✓ All required packages are available!")
except ImportError as e:
    print(f"✗ Missing package: {str(e)}")
    print("IMPORTANT: To install packages, run this command in your terminal (not in the notebook):")
    print("pip install psutil")
    print("Why not install in the notebook?")
    print("- Installing via notebook (!pip install) only affects the current kernel")
    print("- Installing in the terminal ensures packages are available everywhere")
    print("- This ensures consistency across all notebooks and scripts")
    print("After installing, restart the kernel to use the new package.")



✓ All required packages are available!


## Understanding Ports and Network Communication

### Client-Server Communication Diagram

```
                        INTERNET/NETWORK
                              |
                              |
                              v
        +------------------+      Request      +------------------+
        |                  |  -------------->  |                  |
        |     CLIENT       |                   |      SERVER      |
        |                  |  <--------------  |                  |
        +------------------+      Response     +------------------+
               |                                   ^      ^
               |                                   |      |
               v                                   |      |
         Connects to                           Listening on
         specific port                         multiple ports
               |                                   |      |
               v                                   v      v
         +-------------+                     +-------+-------+
         |             |                     |       |       |
         | 192.168.1.5 |                     | :80   | :443  |
         |     :12345  |                     | HTTP  | HTTPS |
         +-------------+                     +-------+-------+
             Client                               Server
            IP:Port                              Ports
```

### Why Port Communication Matters

Port communication is foundational to modern computing for several key reasons:

1. **Enables Multi-Service Architecture**: Ports allow a single device to run multiple network services simultaneously
2. **Service Identification**: Helps identify which application or service should receive incoming data
3. **Security Boundary**: Provides a controlled interface for external systems to access specific services
4. **Network Troubleshooting**: Essential for diagnosing connectivity issues and network problems
5. **Application Development**: Critical knowledge for building distributed applications

### What is a Port?

A port is like a numbered doorway into your computer:
- Each port number (0-65535) can be used for different services
- Common ports: 80 (HTTP), 443 (HTTPS), 22 (SSH), 3306 (MySQL)
- Multiple applications can listen on different ports
- Only one application can use a specific port at a time

### Common Port Ranges:
1. Well-known ports (0-1023)
   - Reserved for standard services
   - Require administrative privileges
   - Examples: HTTP (80), HTTPS (443), FTP (21)

2. Registered ports (1024-49151)
   - Used by applications
   - Don't require special privileges
   - Good for custom applications

3. Dynamic ports (49152-65535)
   - Used for temporary connections
   - Available for any purpose

Let's explore what ports are currently in use on your system:

In [2]:
def list_active_connections():
    """List all active network connections and their ports"""
    connections = psutil.net_connections()
    
    print("Active Network Connections:")
    print("-" * 50)
    print(f"{'Local Address':<20} {'Local Port':<10} {'Status':<10}")
    print("-" * 50)
    
    for conn in connections:
        if conn.laddr:  # Only show connections with local addresses
            addr = conn.laddr.ip
            port = conn.laddr.port
            status = conn.status
            print(f"{addr:<20} {port:<10} {status:<10}")

try:
    list_active_connections()
except Exception as e:
    print(f"Error listing connections: {e}")
    print("Note: Some systems may require elevated privileges to list all connections.")

Active Network Connections:
--------------------------------------------------
Local Address        Local Port Status    
--------------------------------------------------
127.0.0.1            60049      ESTABLISHED
127.0.0.1            9004       LISTEN    
127.0.0.1            60256      ESTABLISHED
::                   56067      NONE      
172.16.10.3          60005      TIME_WAIT 
127.0.0.1            9002       LISTEN    
127.0.0.1            49901      ESTABLISHED
127.0.0.1            49806      ESTABLISHED
172.16.10.3          51462      ESTABLISHED
127.0.0.1            60048      ESTABLISHED
127.0.0.1            4502       LISTEN    
127.0.0.1            53636      ESTABLISHED
127.0.0.1            49672      ESTABLISHED
127.0.0.1            49899      ESTABLISHED
127.0.0.1            49928      ESTABLISHED
::                   5353       NONE      
127.0.0.1            60271      ESTABLISHED
127.0.0.1            9021       LISTEN    
0.0.0.0              49671      LISTEN    

## Creating a TCP Server

Let's create a basic TCP server that:
1. Listens on a specific port
2. Accepts connections from clients
3. Receives and sends messages
4. Properly closes connections

This example will help you understand:
- How servers listen for connections using ports
- How servers handle incoming client connections
- Basic socket programming concepts
- The server side of client-server communication

In [ ]:
def run_tcp_server(host='localhost', port=12345):
    """Run a TCP server that echoes back messages from clients"""
    # Create a socket object
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Allow port reuse (helpful for development)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    
    try:
        # Bind the socket to a specific address and port
        server_socket.bind((host, port))
        
        # Enable the server to accept connections
        # The parameter specifies how many unaccepted connections are allowed
        # before refusing new connections
        server_socket.listen(5)
        
        print(f"Server is listening on {host}:{port}")
        print("Waiting for client connections...")
        print("Stop Cell Execution/Restart the Kernel to stop the server")
        
        # Continue accepting connections until interrupted
        while True:
            # Wait for a client to connect
            client_socket, address = server_socket.accept()
            print(f"Connection established with {address}")
            
            try:
                # Receive data from the client
                data = client_socket.recv(1024).decode('utf-8')
                if not data:
                    print("No data received")
                    continue
                    
                print(f"Received from {address}: {data}")
                
                # Send a response back to the client
                response = f"Server received: {data}"
                client_socket.send(response.encode('utf-8'))
                
            except Exception as e:
                print(f"Error handling client {address}: {e}")
            
            finally:
                # Close the connection
                client_socket.close()
                print(f"Connection with {address} closed")
            
    except KeyboardInterrupt:
        print("\nServer shutdown requested...")
    
    except Exception as e:
        print(f"Server error: {e}")
    
    finally:
        # Always close the server socket
        server_socket.close()
        print("Server stopped")

# Note: Run this function when you're ready to start the server
# It will continue running until you interrupt it (Ctrl+C or stop the kernel)

### Running the TCP Server

To start the TCP server, run the following cell. The server will:
1. Create a socket and bind it to the specified port
2. Listen for incoming connections
3. Accept and process client messages
4. Continue running until interrupted

Once the server is running, you can connect to it using the TCP client notebook.

In [ ]:
# Uncomment and run this cell when you're ready to start the server
run_tcp_server()

Server is listening on localhost:12345
Waiting for client connections...
Press Ctrl+C to stop the server
Connection established with ('127.0.0.1', 60342)
Received from ('127.0.0.1', 60342): Hello from the client notebook!
Connection with ('127.0.0.1', 60342) closed


## Testing Your TCP Server

Once your server is running, try these approaches to verify everything works as expected:

1. Use the companion client notebook (`002_2_tcp_client.ipynb`) to connect and send messages.
2. Use a command-line tool such as `nc` (netcat) from another terminal: `nc localhost 12345`. Type a message and confirm the echoed response.
3. Experiment with multiple connection attempts to see how the server logs different clients.

### Next Steps
- Want to serve web requests? Open the new HTTP server notebook (`002_3_http_server.ipynb`) to build on what you've learned here.
- Continue with the client notebook to understand both sides of the TCP conversation.

Mastering TCP servers provides a strong foundation for deploying AI models as reliable services. Once you are comfortable here, move on to the HTTP-focused materials to see how these ideas power web APIs.
